In [1]:
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import torch
import os
from tqdm import tqdm
from PIL import Image
import random
import tensorflow as tf
import torchvision.transforms.functional as TransformerF
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import transforms, models
import torch.nn.functional as F
import math
from copy import deepcopy
import json
import time



2025-07-01 05:43:02.755119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751348582.778083     235 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751348582.785308     235 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:

CHECKPOINT_PATH = "/kaggle/input/attention-based-multi-module-fusion/data"
BASE_PATH = '/kaggle/input'
m = [
    "wait",
    "bring",
    "friend",
    "good",
    "get_well",
    "get-well",
    "thanks",
    "who",
    "where",
    "why",
    "time",
    "tomorrow",
    "always",
    "apologize",
    "goodbye",
    "single",
    "same",
    "hurry",
    "belt",
    "congratulation",
    "police",
    "single",
    "accident",
    "bed",
    "breakfast",
    "forbidden",
    "sibling",
    "angle",
    "glove",
    "full"
        ]
tester = []
train_video_pathes = []
train_labels = []
for gloss in os.listdir(BASE_PATH):
  if not gloss.__contains__("segmented") :
      continue
  glossName = os.listdir(os.path.join(BASE_PATH,gloss))[0]
  if glossName in m:
    for video_path in os.listdir(os.path.join(BASE_PATH,gloss,glossName)):
        files = os.listdir(os.path.join(BASE_PATH,gloss,glossName,video_path))
        tester.append(video_path)
        train_video_pathes.append([os.path.join(BASE_PATH,gloss,glossName,video_path,files[0]),os.path.join(BASE_PATH,gloss,glossName,video_path,files[1]),os.path.join(BASE_PATH,gloss,glossName,video_path,files[2])])
        train_labels.append(glossName.lower())

BASE_PATH = '/kaggle/input'
for gloss in os.listdir(BASE_PATH):
  if not gloss.endswith("-gloss") or not gloss.split("-gloss")[0] in m :  
      continue
  glossName = gloss.split("-")[0]   
  for video_path in os.listdir(os.path.join(BASE_PATH,gloss)):
    if video_path.endswith("json"):
        continue
    files = os.listdir(os.path.join(BASE_PATH,gloss,video_path))
    tester.append(video_path)
    train_video_pathes.append([os.path.join(BASE_PATH,gloss,video_path,files[0]),os.path.join(BASE_PATH,gloss,video_path,files[1]),os.path.join(BASE_PATH,gloss,video_path,files[2])])
    train_labels.append(glossName.lower())

test_video_pathes = []
test_labels = []
for t in range(1,6):
    BASE_PATH = f'/kaggle/input/test-segmented-continue-part{t}-db/gloss'
    for gloss in os.listdir(BASE_PATH):
      if gloss in m:
        for video_path in os.listdir(os.path.join(BASE_PATH,gloss)):
            files = os.listdir(os.path.join(BASE_PATH,gloss,video_path))
            test_video_pathes.append([os.path.join(BASE_PATH,gloss,video_path,files[0]),os.path.join(BASE_PATH,gloss,video_path,files[1]),os.path.join(BASE_PATH,gloss,video_path,files[2])])
            test_labels.append(gloss.lower())
len(train_video_pathes)

# BASE_PATH = "/kaggle/input/test-continue-part2-db/output"
# for gloss in os.listdir(BASE_PATH):
#   if gloss in m:
#     for video_path in os.listdir(os.path.join(BASE_PATH,gloss)):
#         files = os.listdir(os.path.join(BASE_PATH,gloss,video_path))
#         test_video_pathes.append([os.path.join(BASE_PATH,gloss,video_path,files[0]),os.path.join(BASE_PATH,gloss,video_path,files[1])])
#         test_labels.append(gloss.lower())
len(train_video_pathes)


3243

In [11]:
len(set(train_labels)),len(set(test_labels))

(26, 26)

In [12]:
print(len(tester),len(set(tester)),len(test_labels))
train_dictanory = {t:0 for t in set(np.sort(train_labels))}
test_dictanory = {t:0 for t in set(np.sort(test_labels))}
for t in train_labels:
    train_dictanory[t]+=1
    
for t in test_labels:
    test_dictanory[t]+=1

train_dictanory,test_dictanory

3243 3243 179


({'bed': 127,
  'single': 126,
  'thanks': 126,
  'accident': 125,
  'get_well': 124,
  'tomorrow': 117,
  'hurry': 125,
  'time': 123,
  'apologize': 127,
  'goodbye': 127,
  'breakfast': 125,
  'where': 127,
  'who': 126,
  'belt': 127,
  'wait': 125,
  'glove': 122,
  'why': 127,
  'forbidden': 123,
  'same': 120,
  'full': 127,
  'friend': 121,
  'police': 126,
  'always': 123,
  'bring': 124,
  'sibling': 126,
  'good': 127},
 {'bed': 8,
  'single': 7,
  'thanks': 8,
  'accident': 5,
  'get_well': 6,
  'tomorrow': 7,
  'hurry': 6,
  'time': 7,
  'apologize': 6,
  'goodbye': 7,
  'breakfast': 7,
  'where': 8,
  'who': 8,
  'belt': 6,
  'wait': 6,
  'glove': 7,
  'why': 6,
  'forbidden': 7,
  'same': 7,
  'full': 8,
  'friend': 6,
  'police': 7,
  'always': 8,
  'bring': 8,
  'sibling': 7,
  'good': 6})

In [ ]:
train_dataset = VideoDataset(
    train_video_pathes,
   train_labels,
  agumentation=True,temp_agumentation=False,getCropedImageToo=True
    )
train_dataloader = DataLoader(train_dataset,batch_size=2,shuffle=True,
    num_workers=8,  
    pin_memory=True, 
    prefetch_factor=5,
    timeout=60
                              )
test_dataset = VideoDataset(
    test_video_pathes,
    test_labels,getCropedImageToo=True
    )
test_dataloader = DataLoader(test_dataset,batch_size=2,shuffle=True,
     num_workers=4,  
    pin_memory=True, 
    prefetch_factor=5,
    timeout=60
                             )

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
{val:key for key,val in train_dataset.label_to_index.items()},{val:key for key,val in test_dataset.label_to_index.items()}

({0: 'accident',
  1: 'always',
  2: 'apologize',
  3: 'bed',
  4: 'belt',
  5: 'breakfast',
  6: 'bring',
  7: 'forbidden',
  8: 'friend',
  9: 'full',
  10: 'get_well',
  11: 'glove',
  12: 'good',
  13: 'goodbye',
  14: 'hurry',
  15: 'police',
  16: 'same',
  17: 'sibling',
  18: 'single',
  19: 'thanks',
  20: 'time',
  21: 'tomorrow',
  22: 'wait',
  23: 'where',
  24: 'who',
  25: 'why'},
 {0: 'accident',
  1: 'always',
  2: 'apologize',
  3: 'bed',
  4: 'belt',
  5: 'breakfast',
  6: 'bring',
  7: 'forbidden',
  8: 'friend',
  9: 'full',
  10: 'get_well',
  11: 'glove',
  12: 'good',
  13: 'goodbye',
  14: 'hurry',
  15: 'police',
  16: 'same',
  17: 'sibling',
  18: 'single',
  19: 'thanks',
  20: 'time',
  21: 'tomorrow',
  22: 'wait',
  23: 'where',
  24: 'who',
  25: 'why'})

In [22]:
os.listdir(CHECKPOINT_PATH)

['data.json', 'checkpoint.pth']

In [ ]:
from Model import Model
if not os.path.exists(os.path.join(CHECKPOINT_PATH,'checkpoint.pth')):
    model = Model().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=8e-05)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',         
        factor=0.5,         
        patience=3,         
        verbose=True        
    )
    
    start_epoch= 0 
    test_accuracy = 0
else:
    import torch
    import gc
    from torch import nn
    
    # 1. Force clean GPU memory
    def clear_gpu_memory():
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()  # Reset memory tracking
    
    clear_gpu_memory()
    
    # 2. Load checkpoint differently
    def load_checkpoint_safely(checkpoint_path, device):
        # Load to CPU first
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        
        # Create model on CPU
        model = Model()  # Don't move to GPU yet
        with open(os.path.join(CHECKPOINT_PATH,'data.json'), 'r') as file:
            data = json.load(file)
            
        if 'best test accuracy' in data:
            test_accuracy = data["best test accuracy"]
        else:
            test_accuracy = 0
         # Load state dict in eval mode (uses less memory)
        model.eval()
        model.load_state_dict(checkpoint['best_model_state_dict'])
        
        # Now move to GPU
        model = model.to(device)
        
        # Create optimizer after model is on GPU
        optimizer = torch.optim.AdamW(model.parameters(), lr=8e-05)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        return {
            'model': model,
            'optimizer': optimizer,
            'epoch': checkpoint['epoch'],
            'scheduler_state': checkpoint.get('scheduler_state_dict', None)
        },test_accuracy
    

    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Load checkpoint
    loaded, test_accuracy = load_checkpoint_safely(os.path.join(CHECKPOINT_PATH,'checkpoint.pth'), device)
    
    model = loaded['model']
    optimizer = loaded['optimizer']
    start_epoch = loaded['epoch']
    
    # Recreate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3,
        verbose=True
    )
    
    if loaded['scheduler_state'] is not None:
        scheduler.load_state_dict(loaded['scheduler_state'])
                
classification_loss = nn.CrossEntropyLoss ()

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [24]:
print("started epoch: ", start_epoch)
print("test accuracy: ",test_accuracy)

started epoch:  82
test accuracy:  0.8938547486033519


In [ ]:
from tools import train
history = train(model,classification_loss_fn,epoch,train_loader,test_loader,scheduler,test_accuracy,CHECKPOINT_PATH,optimizer,device,train_dataset)